# **Loading Images Using `tf.data.Dataset`**

**Learning objectives**
1. Retrieve images using `tf.keras.utils.get_file`
2. Load images using Keras Pre-Processing
3. Load images using `tf.data.Dataset`
4. Understand basic methods for training

## **Introduction**